<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Neural-Nets-Are-Not-Black-Boxes" data-toc-modified-id="Neural-Nets-Are-Not-Black-Boxes-1">Neural Nets Are Not Black Boxes</a></span></li><li><span><a href="#Objective" data-toc-modified-id="Objective-2">Objective</a></span></li><li><span><a href="#Back-propagation" data-toc-modified-id="Back-propagation-3">Back-propagation</a></span></li><li><span><a href="#Binary-Cross-Entropy" data-toc-modified-id="Binary-Cross-Entropy-4">Binary Cross Entropy</a></span></li><li><span><a href="#Activations" data-toc-modified-id="Activations-5">Activations</a></span></li><li><span><a href="#Linear-Layer" data-toc-modified-id="Linear-Layer-6">Linear Layer</a></span></li><li><span><a href="#Putting-It-All-Together" data-toc-modified-id="Putting-It-All-Together-7">Putting It All Together</a></span></li><li><span><a href="#Our-Evaluation-Metric" data-toc-modified-id="Our-Evaluation-Metric-8">Our Evaluation Metric</a></span></li><li><span><a href="#Trainer" data-toc-modified-id="Trainer-9">Trainer</a></span></li><li><span><a href="#Pre-process-Data" data-toc-modified-id="Pre-process-Data-10">Pre-process Data</a></span></li><li><span><a href="#Datasets-&amp;-DataLoaders" data-toc-modified-id="Datasets-&amp;-DataLoaders-11">Datasets &amp; DataLoaders</a></span></li><li><span><a href="#Train" data-toc-modified-id="Train-12">Train</a></span></li><li><span><a href="#Comparison-with-sklearn" data-toc-modified-id="Comparison-with-sklearn-13">Comparison with sklearn</a></span></li><li><span><a href="#Adding-More-Layers" data-toc-modified-id="Adding-More-Layers-14">Adding More Layers</a></span></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-15">Conclusion</a></span></li></ul></div>

# Neural Nets Are Not Black Boxes 

If you think neural nets are black boxes, you're certainly not alone. While they may not be as interpretable as models like random forests (at least not yet), we can still understand how networks process data to arrive at their predictions, and that's exactly what we'll do in this post. We'll build our own network from scratch, starting with logistic regression.

This post is very much inspired by [this fantastic post](https://sgugger.github.io/a-simple-neural-net-in-numpy.html#a-simple-neural-net-in-numpy) by Sylvain Gugger. We won't pretend to improve upon Sylvain's post; we just want to explain things in our own way to help us understand things a little bit more clearly. This will be the first of a series of posts in which we'll write our own DNN, CNN, and RNN. You can find the source code for all of these posts at [tinytorch](https://github.com/msarmi9/tinytorch).

In [1]:
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

In [2]:
# Set seed for reproducibility
seed = 9
np.random.seed(seed)

# Objective 

Our goal is to construct a binary logistic classifier as a neural network. The network will consist of a single linear layer followed by a sigmoid activation with binary cross entropy as the loss. We'll begin by deriving the back-prop equations for our particular scenario and in doing so we'll see that what we've done generalizes immediately to networks with arbitrary layers and activations. In other words, we'll have developed a framework that can model any feedforward network--all by starting from ordinary logistic regression. 

Actually, this isn't all that surprising when you think about it. Logistic regression is a linear layer followed by sigmoid and feedforward networks are just a bunch of linear layers stacked together with non-linear activations in between.

# Back-propagation

Back-propagation is nothing more than the chain rule. We can view our logistic network as the composition of three functions

$$x \to \text{BCE} \circ \text{Sigmoid} \circ \text{Linear}(x)$$

While the loss function is not usually viewed as a layer of the network, treating it as the final layer makes computing the gradients easier. Let's denote the output of the $i$-th layer by $x_i$ so that

\begin{align}
    x_1 &= \text{Linear}(x)     \\
    x_2 &= \text{Sigmoid}(x_1)  \\
    x_3 &= \text{BCE}(x_2)
\end{align}

The first gradient we have to compute is the gradient of $\text{BCE}$ with respect to the activations $x_2$.

$$\frac{\partial \text{BCE}}{\partial x_2} = \frac{\partial \text{BCE}}{\partial x_2}(x_2) $$

Next we have to compute the gradient with respect to the linear outputs $x_1$. The chain rule tells us

$$\frac{\partial \text{BCE}}{\partial x_1} = \frac{\partial \text{BCE}}{\partial x_2} \times \frac{\partial \text{Sigmoid}}{\partial x_1}(x_1)$$

Last, we'll need to compute the gradient with respect to the original inputs $x$

$$\frac{\partial \text{BCE}}{\partial x} = \frac{\partial \text{BCE}}{\partial x_1} \times \frac{\partial \text{Linear}}{\partial x_1}(x)$$

Notice a pattern? The first gradient we computed--the gradient with respect to the network's final activations--is used to compute the next gradient--the gradient with respect to the linear outputs--which are in turn used to compute the gradient with respect to the original inputs. To compute the gradients of any network, we simply start at the last layer and successively pass the gradients backwards to the preceding layer until we arrive at the original inputs. That's the reason it's called back-propagation. It really is helpful to picture passing the gradients backwards through the network like a baton.

We'll compute each of these gradients in turn, starting with the last layer and working our way backwards to the original inputs.

__Note:__ So far we've been treating the input $x$ as a single variable, but most of the time $x$ will have more than one dimension. Computing the gradients in the multi-variate case isn't anymore difficult than what we've done (it involves something called the Jacobian, but we'll pretend we didn't hear that).

--------

# Binary Cross Entropy

Binary cross entropy penalizes predictions by the logarithm of their confidence. Given labels $y$ which are either zero or one and probabilities $\hat{y}$ for the positive class, we add $-\ln(\text{P}(y=1))$ to the loss whenever $y = 1$ and $-\ln(\text{P}(y=0))$ whenever $y = 0$. In other words,

$$\text{BCE}(\hat{y}, y) = -[y \ln(\hat{y}) + (1 - y)\ln(1 - \hat{y})]$$

After simplifying, you'll find its derivative is

$$\frac{\partial \text{BCE}}{\partial \hat{y}} = \frac{\hat{y} - y}{\hat{y}(1 - \hat{y})}$$

To avoid potential division-by-zero errors, we'll clip the probabilities $\hat{y}$ so that they're not too close to zero or one.

In [3]:
class BinaryCrossEntropy:
    """Container for the forward and backward pass of BCE."""
    
    def forward(self, y_hat, y):
        """Return binary cross entropy given predictions and targets."""
        self.y_hat, self.y = y_hat.clip(min=1e-8, max=1-1e-8), y
        return -np.where(y==1, np.log(self.y_hat), np.log(1 - self.y_hat))
    
    def backward(self):
        """Backpropagate the gradient with respect to predictions."""
        return (self.y_hat - self.y) / (self.y_hat * (1 - self.y_hat))

-----

# Activations

The easiest components of networks to handle are the activation functions. Our activation is sigmoid, which you'll often see defined as one of 

$$\sigma(x) = \frac{1}{1 + \text{exp}(-x)} \quad \text{or} \quad \frac{\text{exp}(x)}{1 + \text{exp}(x)}$$

It turns out we need both versions to implement a numerically stable sigmoid. Why? Notice how when $x$ is very negative, $\text{exp}(-x)$ is very large, and when $x$ is very positive, $\text{exp}(x)$ is very large--in both cases too large to store in memory. The easy fix is to use the former when $x > 0$ and the latter when $x < 0$.

After simplifying, you'll find the derivative of sigmoid is

$$\sigma'(x) = \sigma(x)(1 - \sigma(x))$$

Notice something interesting? Since we denoted $\hat{y}$ above as the output of sigmoid, this is exactly the denominator of the BCE gradient we just computed, meaning the two terms will cancel when we compute the gradient with respect to the outputs $x_1$ of our network's last (and only) linear layer. After canceling, we're left with

$$\frac{\partial \text{BCE}}{\partial x_1} = \hat{y} - y $$

Nice, right? This tells us that the gradient of the loss with respect to the network's final linear outputs is just the difference between the probabilities $\hat{y}$ and the labels $y$. The further apart they are (i.e. the worse our predictions are), the larger the gradient and the larger the update to the last linear layer's weights in the SGD step (remember, the chain rule tells us the above gradient appears as a factor in the gradient with respect to the weights of the last linear layer).

This is terrific because it means the weights of our network will change gradually as we train and won't spike or drop suddenly, which would be the case if the gradients were a quadratic or higher-order function of the prediction error. It also demonstrates nicely how a network adjusts its weights based on the error of its predictions.

In fact, the same is true of multi-classification, where instead of sigmoid we use softmax (or log softmax) as the final activation and cross entropy as the loss. In this case, the gradient with respect to the last linear layer's weights $x$ is

$$
\frac{\partial \text{CE}}{\partial x}(\hat{y}, y) = \frac{\partial \text{CE}}{\partial \hat{y}}(\hat{y}) \times \frac{\partial \text{softmax}}{\partial x}(x) = \hat{y} - y
$$

We might cover the multi-class case in more depth in a follow-up post, although it's more or less the same as what we've done so far.

In [4]:
class Sigmoid:
    """Container for the forward and backward pass of sigmoid."""
    
    def forward(self, x):
        """Pass a mini-batch through a sigmoid layer."""
        self.y_hat = np.where(x > 0, 1 / (1 + np.exp(-x)), np.exp(x) / (1 + np.exp(x)))
        return self.y_hat
        
    def backward(self, grad):
        """Backpropagate the gradient given the preceding gradient."""
        return self.y_hat * (1 - self.y_hat) * grad

----------

# Linear Layer

The last and most difficult component we need to implement is the linear layer, which contains weights and biases. Denoting the linear outputs by $z$, we have

$$z = xw + b$$

If $x$ is a mini-batch of shape $(bs, n_{inp})$, then $w$ has shape $(n_{inp}, 1)$ and $b$ has shape $(1,)$, with addition being done via broadcasting. To make things easier, for the moment let's just imagine we have a batch size of one.

$$x = [x_1, \dots, x_{n_{inp}}]$$ 

There are two gradients to compute this time around, one with respect to the weights and another with respect to the bias. To make life easier still, let's write everything out in coordinates.

$$z_i = \sum_{k=1}^{n_{inp}} x_k w_{ki} + b$$

Taking the derivative with respect to the weights, we get

$$\frac{\partial \text{BCE}}{\partial w_{ki}} = \frac{\partial \text{BCE}}{\partial z_i} \times \frac{\partial z_i}
{\partial w_{ki}} = \frac{\partial \text{BCE}}{\partial z_i} \times x_k$$

Taking a closer look at these gradients, we see exactly why neural nets are so sensitive to the scale of their inputs. Because the gradients with respect to the weights $w$ are scaled by the input features $x$, having features of different magnitudes will result in some gradients being larger than others. These larger gradients will dominate the learning process and prevent the network from learning from all features equally. This is why it's important to always normalize your data before training.

For the derivative with respect to the bias, we have

$$\frac{\partial \text{BCE}}{\partial b} = \frac{\partial \text{BCE}}{\partial z_i} \times \frac{\partial z_i}{\partial b} = \frac{\partial \text{BCE}}{\partial z_i}$$

Notice something nice? Since we'll already have the gradient with respect to the linear outputs $z$ stored beforehand in a variable called $\text{grad}$, we get the gradient with respect to the bias for free, leaving just the weights to deal with. The main obstacle is figuring out how to write the above equations as a matrix product. Whenever I have to do something like this, I just focus on getting the shapes right.

\begin{align}
  &\bullet x \text{ has shape } (bs, n_{inp}) \\
  &\bullet \text{grad has shape } (bs, 1) \\
  &\bullet \text{grad}_W \text{ has shape } (n_{inp}, 1)
\end{align}

The only way we can multiply $x$ and $\text{grad}$ and get something of shape $(n_{inp}, 1)$ is to re-shape $x$ to have shape $(bs, n_{inp}, 1)$ and $\text{grad}$ to have shape $(bs, 1, 1)$ so that ordinary matrix multiplication over the last two dimensions gives the shape $(n_{inp}, 1)$.

Were there another linear layer we'd also need to compute the gradient with respect to the inputs $x$ so we could keep back-propagating the gradients. This isn't anymore complicated than what we've done so far and doing so will allow us to build a network with any number of layers, so let's go ahead and do it. Since $x_k$ appears in each of the activations $z_i$, the gradient will respect to $x_k$ will involve summing all of the intermediate gradients with respect to $z_i$.

$$ \frac{\partial \text{BCE}}{x_k} = \sum_{i=1}^{n_{inp}} \frac{\partial \text{BCE}}{\partial z_i} \times \frac{\partial z_i}{\partial x_k} = \sum_{i=1}^{n_{inp}} \frac{\partial \text{BCE}}{\partial z_i} w_{ki}$$

We can re-write this as a matrix product using the transpose of the weight matrix.

$$ \frac{\partial \text{BCE}}{\partial x_k} = \text{grad} \times W^t$$

Let's do a sanity check on the dimensions involved to make sure nothing has gone horribly wrong. Since $\text{grad}$ has shape $(bs, 1)$ and $W$ has shape $(n_{inp}, 1)$, $\text{grad} \times W^t$ has shape $(bs, n_{inp})$, which is exactly the shape of $x$--just as it should be.

In [5]:
class Linear:
    """Container for the forward and backward pass of a linear layer."""
    
    def __init__(self, n_inp, n_out):
        """Initialise layer with random weights and zero bias."""
        k = 1 / np.sqrt(n_inp)
        self.weights = np.random.uniform(-k, k, (n_inp, n_out))
        self.bias = np.zeros(n_out)
        
    def forward(self, x):
        """Pass a mini-batch through a linear layer."""
        self.x = x
        return x @ self.weights + self.bias
    
    def backward(self, grad):
        """Backpropagate the gradient given the preceding gradient."""
        self.grad_w = (self.x[:,:,None] @ grad[:,None,:]).mean(axis=0)
        self.grad_b = grad.mean(axis=0)
        return grad @ self.weights.T

---------

# Putting It All Together

It's finally time to string together all of the work we've done so far into a complete network. Then we'll put it to the test on the [breast cancer dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html#sklearn.datasets.load_breast_cancer) and see how we compare to sklearn's built-in logistic model. 

In [6]:
class Sequential:
    """Container for a feedforward neural net."""
    
    def __init__(self, layers, criterion):
        """Initialise layers and loss criterion."""
        self.layers = layers
        self.criterion = criterion
        
    def forward(self, x):
        """Pass a mini-batch through the network."""
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def backward(self):
        """Backpropagate gradients to the start of the network."""
        grad = self.criterion.backward()
        for layer in self.layers[::-1]:
            grad = layer.backward(grad)

In [7]:
class SGD:
    """Container for updating a model's weights via SGD."""
    
    def __init__(self, model, lr):
        """Initialise model parameters and learning rate."""
        self.model = model
        self.lr = lr
                  
    def step(self):
        """Update weights and biases of all linear layers."""
        for layer in self.model.layers:
            if isinstance(layer, Linear):
                layer.weights -= self.lr * layer.grad_w
                layer.bias -= self.lr * layer.grad_b

----------

# Our Evaluation Metric

For simplicity, we'll just consider accuracy as our evaluation metric for the time being.

In [8]:
def accuracy(y_hat, y):
    """Compute accuracy given soft binary predictions."""
    y_pred = y_hat > 0.5
    return (y_pred == y).mean()

---------

# Trainer

To make life easier, let's wrap all of the functionality we'll need to train a network in a single class.

In [9]:
class Trainer:
    """Container for training a feedforward neural net."""
    
    def __init__(self, model, optimizer, train_dl, val_dl, metric):
        self.model = model
        self.optimizer = optimizer
        self.train_dl = train_dl
        self.val_dl = val_dl
        self.metric = metric
        
    def train_one_epoch(self):
        """Train for one epoch and return the loss."""
        loss, n = 0, 0
        for x, y in self.train_dl:
            y_hat = self.model.forward(x)
            batch_loss = self.model.criterion.forward(y_hat, y).sum()
            self.model.backward()
            self.optimizer.step()
            loss += batch_loss
            n += len(y)
        return loss / n
            
    def train(self, n_epochs, log_level=1):
        """Train for several epochs."""
        for epoch in range(n_epochs):
            loss = self.train_one_epoch()
            val_loss, val_metric = self.evaluate(self.val_dl)
            if (epoch + 1) % log_level == 0:
                print(f"{epoch= :2d} | {loss= :.3f} | {val_loss= :.3f} | {val_metric= :.3f}")
    
    def evaluate(self, dl):
        """Return loss and metric on validation or test set."""
        loss, n, metric = 0, 0, 0
        for x, y in dl:
            y_hat = self.model.forward(x)
            batch_loss = self.model.criterion.forward(y_hat, y).sum()
            batch_metric = self.metric(y_hat, y)
            metric += len(y) * batch_metric
            loss += batch_loss
            n += len(y)
        return loss / n, metric / n

---------

# Pre-process Data

We'll use sklearn's [breast cancer dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html#sklearn.datasets.load_breast_cancer) for our binary classification task. 

In [10]:
# Load data
X, y = load_breast_cancer(return_X_y=True)
X.shape, y.shape

((569, 30), (569,))

In [11]:
# Train-test-split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=seed)
X_train.shape, X_val.shape

((455, 30), (114, 30))

In [13]:
def normalize(X_train, X_val):
    """Normalize training and validation data using training stats."""
    for j in range(X_train.shape[1]):
        mu, sigma = X_train[:,j].mean(), X_train[:,j].std()
        X_train[:,j] = (X_train[:,j] - mu) / sigma
        X_val[:,j] = (X_val[:,j] - mu) / sigma
    return X_train, X_val

In [14]:
# Normalize with training stats
X_train, X_val = normalize(X_train, X_val)

---------

# Datasets & DataLoaders

In order to train in batches, we'll need to implement our own version of pytorch's datasets and dataloaders, since we're doing everything in numpy.

In [16]:
class Dataset:
    """Container for returning inputs and targets."""
    
    def __init__(self, X, y):
        """Initialise inputs and re-shape targets as a column vector."""
        self.X = X
        self.y = y.reshape(-1, 1)
        
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]
    
    def __setitem__(self, idx, val):
        self.X[idx], self.y[idx] = val
                
    def __len__(self):
        return len(self.y)

In [17]:
class DataLoader:
    """Container for returning a mini-batch of inputs and targets."""
    
    def __init__(self, ds, batch_size, shuffle=False):
        """Initialise dataset and batch size."""
        self.ds = ds
        self.batch_size = batch_size
        self.shuffle = shuffle
        
    def shuffle_data(self):
        """Shuffle inputs and targets."""
        idxs = np.random.permutation(len(self.ds))
        self.ds = Dataset(*self.ds[idxs])
        
    def __iter__(self):
        """Yield a mini-batch of inputs and targets."""
        if self.shuffle: self.shuffle_data()
        n_batches = len(self.ds) // self.batch_size
        for i in range(n_batches):
            yield self.ds[i * self.batch_size: (i + 1) * self.batch_size]

In [18]:
# Load training and validation data
train_ds = Dataset(X_train, y_train)
val_ds = Dataset(X_val, y_val)

batch_size = 64
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=len(X_val), shuffle=False)

--------

# Train

Now we're ready to put our model to the test.

In [25]:
# Input and final output dims
n_inp = X_train.shape[1]

# Initialise layers and criterion
metric = accuracy
criterion = BinaryCrossEntropy()
layers = [Linear(n_inp, 1), Sigmoid()]
model = Sequential(layers, criterion)

# Initialise optimizer and trainer
optimizer = SGD(model, lr=0.1)
trainer = Trainer(model, optimizer, train_dl, val_dl, metric)

In [27]:
trainer.train(10)

epoch=  0 | loss= 0.111 | val_loss= 0.106 | val_metric= 0.965
epoch=  1 | loss= 0.108 | val_loss= 0.104 | val_metric= 0.965
epoch=  2 | loss= 0.104 | val_loss= 0.102 | val_metric= 0.974
epoch=  3 | loss= 0.102 | val_loss= 0.100 | val_metric= 0.974
epoch=  4 | loss= 0.099 | val_loss= 0.098 | val_metric= 0.974
epoch=  5 | loss= 0.098 | val_loss= 0.096 | val_metric= 0.982
epoch=  6 | loss= 0.097 | val_loss= 0.095 | val_metric= 0.982
epoch=  7 | loss= 0.095 | val_loss= 0.094 | val_metric= 0.982
epoch=  8 | loss= 0.094 | val_loss= 0.092 | val_metric= 0.982
epoch=  9 | loss= 0.091 | val_loss= 0.091 | val_metric= 0.982


-------------

# Comparison with sklearn

Let's see how our logistic network stacks up against sklearn's logistic classifier.

In [21]:
# We're close!
sklearn_model = LogisticRegression(random_state=seed)
sklearn_model.fit(X_train, y_train)
sklearn_model.score(X_val, y_val)

0.9824561403508771

----------------

# Adding More Layers

Now let's see if we can do a bit better by training a deeper network. First, we'll need to implement $\text{ReLU}$ for the activations in between our linear layers. $\text{ReLU}$ functions as a gate that only admits positive inputs and sends all negative inputs to zero.

$$
\begin{align}
    \text{ReLU}(x) = \begin{cases} x & \text{if } x > 0 \\ 0 & \text{otherwise} \end{cases}
\end{align}
$$

You might recognize its derivative as the Heaviside function, which models an electric current that's turned on at time $t = 0$.

\begin{align}
    \text{ReLU}'(x) = \begin{cases} 1 & \text{if } x > 0 \\ 0 & \text{otherwise} \end{cases}
\end{align}

During the backward pass $\text{ReLU}$ again functions as a gatekeeper that only admits gradients corresponding to positive inputs and sends all other gradients to zero. 

In [22]:
class ReLU:
    """Container for the forward and backward pass of ReLU."""
    
    def forward(self, x):
        """Pass a mini-batch through ReLU."""
        self.x = x
        return np.where(x > 0, x, 0)
    
    def backward(self, grad):
        """Return the gradient where x is positive, otherwise zero."""
        return np.where(self.x > 0, grad, 0)

In [28]:
# Input and final output dims
n_inp = X_train.shape[1]

# Initialise layers and criterion
metric = accuracy
criterion = BinaryCrossEntropy()
layers = [Linear(n_inp, 20), ReLU(), Linear(20, 1), Sigmoid()]
model = Sequential(layers, criterion)

# Initialise optimizer and trainer
optimizer = SGD(model, lr=0.10)
trainer = Trainer(model, optimizer, train_dl, val_dl, metric)

In [34]:
trainer.train(10)

epoch=  0 | loss= 0.054 | val_loss= 0.076 | val_metric= 0.974
epoch=  1 | loss= 0.052 | val_loss= 0.076 | val_metric= 0.974
epoch=  2 | loss= 0.053 | val_loss= 0.076 | val_metric= 0.974
epoch=  3 | loss= 0.052 | val_loss= 0.076 | val_metric= 0.974
epoch=  4 | loss= 0.052 | val_loss= 0.076 | val_metric= 0.974
epoch=  5 | loss= 0.046 | val_loss= 0.076 | val_metric= 0.974
epoch=  6 | loss= 0.050 | val_loss= 0.076 | val_metric= 0.974
epoch=  7 | loss= 0.051 | val_loss= 0.076 | val_metric= 0.974
epoch=  8 | loss= 0.051 | val_loss= 0.076 | val_metric= 0.982
epoch=  9 | loss= 0.050 | val_loss= 0.076 | val_metric= 0.982


-------------

# Conclusion

That's all for today. Thanks for reading!